In [36]:
import os
import pandas as pd
import numpy as np

In [37]:
# load the japanese phone dictionary 
def load_phone_dict(path):
  phone_dict = list()
  with open(path, 'r') as f:
    for line in f:
      phone_dict.append(line.strip())
  return phone_dict  

In [38]:
def phone_to_one_hot(phone, phone_dict):
  tmp = list(np.zeros(len(phone_dict)))
  if phone=='None':   
    return tmp
  else:
    tmp[phone_dict.index(phone)] = 1.0
    return tmp

In [39]:
def time_to_frame(time, window_size, step_size):
  number_windows=(time-2*window_size)//step_size
  if number_windows >= 0:
    return number_windows
  else:
    return 0

In [67]:
#### convert /mono/.labs to csv
#### a floder under /mono/ called /csv/ will be created and it will hold all the csv files with current, next, before
#### the format of one example will be shown below

label_path = '/Users/ShiyuMu/Desktop/HTS-demo_NIT-SONG070-F001/data/labels/mono/'
fileList = [label_path + f for f in os.listdir(label_path) if f.endswith('.lab')]
try:
  # create dictionary to hold all the csv file 
  os.mkdir(label_path + 'csv/');
  print('csv dictionary created')
except:
  pass
for file in fileList:
  df = pd.read_csv(file, sep=" ", header=None)
  df.columns = ["beg", "end", "current"]
  df['before'] = pd.Series(['None']).append(df['current']).reset_index(drop=True)
  df['next'] = df['current'][1:].append(pd.Series(['None'])).reset_index(drop=True)
  df.to_csv(label_path + 'csv/'+ file.split('/')[-1]+'.csv', sep='\t', index=False)

csv dictionary created


### example of loading csv files 

In [68]:
## this is the path we just created
csv_path = label_path + 'csv/'
# get all csv files 
csvList = [csv_path + f for f in os.listdir(csv_path) if f.endswith('.csv')]
df_list = list()
## add all csv files as dataframe to a list
for file in csvList:
  df = pd.read_csv(file, sep='\t')
  df_list.append(df)

In [70]:
## the df_list contains 31 dataframe (since we have 31 dataset)
## one dataframe will look like this. beg and end here indicates nonoseconds
df_list[0].head()

,beg,end,current,before,next
0,0,23800000,pau,None,n
1,23800000,24400000,n,pau,e
2,24400000,30450000,e,n,N
3,30450000,35800000,N,e,n
4,35800000,36200000,n,N,e


### load phoneme dictionary

In [47]:
phone_dict_path = 'JPN_phone_dict.txt'
phone_dict = load_phone_dict(phone_dict_path)

### convert timestamps to frame numbers

In [49]:
#### define window size and step size to convert timestamp to frame id
window_size = 250000
step_size = 100000

for df in df_list:
#   print(len(df['beg'].values))
  for index in range(len(df['beg'].values)):
#     print(index)
    df['beg'].values[index] = time_to_frame(df['beg'].values[index], window_size, step_size)
  for index in range(len(df['end'].values)):
#     print(index)
    df['end'].values[index] = time_to_frame(df['end'].values[index], window_size, step_size)

In [50]:
## now df_list contains 31 dataframe, one would be like this
df_list[0].head()

,beg,end,current,before,next
0,0,233,pau,None,n
1,233,239,n,pau,e
2,239,299,e,n,N
3,299,353,N,e,n
4,353,357,n,N,e


### Unfold all frames and list them one by one

In [26]:
unfolded_df_list = list()
for dataset in df_list:
  unfold_list = list()
  for index in range(dataset.shape[0]):
    beg_ = dataset.iloc[index]['beg']
    end_ = dataset.iloc[index]['end']
    current_ = dataset.iloc[index]['current']
    before_ = dataset.iloc[index]['before']
    next_ = dataset.iloc[index]['next']
    if index == 0:
      unfold_list.append(['None', current_, next_])
    else:
      unfold_list.append([before_, current_, next_])
    for i in range(beg_+1, end_):
      unfold_list.append([before_, current_, next_])
    if index == (dataset.shape[0] -1):
      unfold_list.append([before_, current_, 'None'])
    else:
      unfold_list.append([before_, current_, next_])
  unfold_df = pd.DataFrame(unfold_list, columns=['before', 'current', 'next'])
  unfolded_df_list.append(unfold_df)

In [53]:
## now unfolded_df_list is a list contains 31 dataframe, 
## one example would be like this, the index at the front means frame id
unfolded_df_list[0].head()

,before,current,next
0,None,pau,n
1,None,pau,n
2,None,pau,n
3,None,pau,n
4,None,pau,n


### save the unfolded dataframe to disk

In [71]:
### a folder under /mono/csv/ called /unfolded/ will becreated which holds all datafames like above
try:
  # create dictionary to hold all the unfolded file 
  os.mkdir(label_path + 'csv/unfolded/');
  print('unfolded folder created')
except:
  print('folder already exist')
  pass
for i in range(len(csvList)):
  unfolded_df_list[i].to_csv(label_path + 'csv/unfolded/'+ csvList[i].split('/')[-1], sep='\t', index=False)

unfolded folder created


### After you created all the files, you can load them back to memory easily


## NOTICE, if you already have all the files saved, the code below is the only thing you need to load the file and start to work with your training. The code before is only for generating. 

In [72]:
## this is the path we just created
unfolded_path = label_path + 'csv/unfolded/'
# get all csv files 
fileNameList = [unfolded_path + f for f in os.listdir(unfolded_path) if f.endswith('.csv')]
unfolded_df_list_ = list()
## add all csv files as dataframe to a list
for file in fileNameList:
  df = pd.read_csv(file, sep='\t')
  unfolded_df_list_.append(df)

In [73]:
unfolded_df_list_[0].head()

,before,current,next
0,None,pau,n
1,None,pau,n
2,None,pau,n
3,None,pau,n
4,None,pau,n


### of course you may also want the filename of this specify dataframe to match with raw files

In [74]:
# the index of fileNameList maintains the same with unfolded_df_list_
fileNameList[0].split('/')[-1]

'nitech_jp_song070_f001_010.lab.csv'

### get one-hot-vector, call phone_to_one_hot() to a specific phoneme

In [75]:
one_hot = phone_to_one_hot('sil', phone_dict)
print('sil', one_hot)

sil [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


### to access one specifiy position

In [77]:
### the first dataframe in the list, the index 0, and the current label
unfolded_df_list_[0].iloc[0]['current']

'pau'